In [ ]:
#Import numpy so that we can actually do maths ...
import numpy as np



#We define the measurement function to compute (in absence of noise) the measured values.
#This particular measurement function is based on a spherical sound-amplitude attenuation model.
def Measure(Prediction):
    r = np.linalg.norm(S.T - ([(Prediction.T[0])[0],(Prediction.T[0])[1]]) , axis = 1)
    Mea = rho - (5/np.log(10))*np.log(1 + r*r)
    #If the 'measured' value would no be greater than the background noise (set to be 0),
    # the measurement function returns zero. 
    Mea = np.array([Mea]).T
    return(Mea)



#We also define a function to compute the Jacobian of the measurement transformation.
def Jacobian(Sensors , Prediction):
    Jac = np.zeros([p,4])
    for j in range(0,p):
        r = np.linalg.norm(np.array([(Prediction.T[0])[0],(Prediction.T[0])[1]]) - Sensors.T[j])
        Jac[j,0] = (-10/np.log(10))*(Prediction[0] - (Sensors.T[j])[0])/(1 + r*r)
        Jac[j,1] = (-10/np.log(10))*(Prediction[1] - (Sensors.T[j])[1])/(1 + r*r)
    return(Jac)



#Extended Kalman Filter for single-target tracking:

#Inputs:
# - InitVal - the 'true' initial values of the state vector (x, y, x-dot, y-dot).
# - X0 - an initial estimate for the state vector.
# - P0 - an estimated variance/uncertainty of the initial estimate.
# - p - the number of sensors
# - S - a 2-by-p matrix with the sensors' x-coordinates in the first row, and y-coordinates in the second

def EKF(InitVal, X0, P0, p, S):
    
    #Parameters:
    # - Dt - the time interval between iterations
    Dt = 1
    # - A - the state-state transition matrix; this A assumes a linear (approximation to the) evolution equation:
    A = np.eye(4)
    A[0,2] = Dt
    A[1,3] = Dt
    # - MeasErr - estimated measurement error, used as the variance of the (simulated) measurements' distribution
    MeasErr = 0.5
    M = MeasErr
    # - rho -
    rho = 20
    # - N - a maximum number of iterations to perform.
    N = 100
    # - cutoff - a number of consecutive ''no measurement''s after which the filter will stop
    cutoff = 5
    
    #Initialise arrays to record the estimated and 'measured' values.
    Estim = np.zeros((4,N+1))
    Measu = np.zeros((p,N))
    Trace = np.zeros((1,N+1))
    Sigma = np.zeros((1,N+1))
    Y = np.zeros((p,1))
    
    #Start an iteration counter.
    i = 0
    
    #Write the initial estimate into the record.
    Estim.T[0] = X0.T
    Trace.T[0] = np.trace(P0)
    Sigma.T[0] = P0[0,0] + P0[1,1]
    
    #Start the process evolution
    TrueVal = A@InitVal
    
    #Start the filter
    
    #X -- State matrix
    X = X0
    
    #P -- State covariance matrix
    P = P0
    
    #R -- Measurement covariance matrix
    R = M*M*np.eye(p)
    
    zerocount = 0
    
    while (i < N)&(zerocount < cutoff):
        #Xp -- Predicted state
        Xp = A@X                        
        
        #Pp -- Predicted state covariance
        Pp = A@P@A.T
        
        #Y -- 'Measured' value
        Y = np.maximum( 0 , Measure(TrueVal) + np.array([np.random.multivariate_normal(np.zeros(p),R)]).T)
        if np.linalg.norm(Y) == 0:
            zerocount = zerocount + 1
        else:
            zerocount = 0    
        
        Measu.T[i] = Y.T
        H = Jacobian(S,Xp) * (1*(Y > 0))
        # We zero out the Jacobian entries corresponding to zero measurements
        
        #K -- Kalman gain
        K = Pp@H.T@np.linalg.inv(H@Pp@H.T + R)
        
        #Update X and P
        X = Xp + K@(Y - Measure(Xp))
        P = (np.eye(4) - K@H)@Pp
        
        #Increment the iteration counter
        i = i+1
        
        #Write the new values to the record.
        Estim.T[i] = X.T
        Trace.T[i] = np.trace(P)
        Sigma.T[i] = P[0,0] + P[1,1]
        
        #Update the 'true' state
        TrueVal = A@TrueVal
    
    
    # Remove extraneous record entries:
    if zerocount == cutoff:
        n = i - cutoff
        Measu = np.delete(Measu,np.arange(N-n)+n,1) # This is now a p-by-n array
        Estim = np.delete(Estim,np.arange(N-n)+n+1,1) # This is now a 4-by-(n+1) array
        Trace = np.delete(Trace,np.arange(N-n)+n+1,1) # This is now a 1-by-(n+1) array
        Sigma = np.delete(Sigma,np.arange(N-n)+n+1,1) # This is now a 1-by-(n+1) array
    else:
        n = N
    
    return(n, Measu, Estim, Trace, Sigma)

# Outputs:
# - n - the number of iterations performed
# - Measu - the sound measurements from the p sensors, stored as the entries of a p-by-n array
# - Estim - the sequential state-vector estimates, stored as the columns of a 4-by-(n+1) array
# - Trace - the traces of the sequential state covariance matrices, stored as the entries of a 1-by-(n+1) array
# - Sigma - the sum of the position variances (the first two diagonal entries of the state covariance matrices),
#            stored as the entries of a 1-by-(n+1) array